In [1]:
import pandas as pd

In [2]:
df_en = pd.read_csv('wiki_en_jpeg.csv', usecols=["image_url", "caption_reference_description"])

In [3]:
df_en = df_en.rename(columns={'caption_reference_description' : 'caption_description'})

In [13]:
import requests
from requests.sessions import Session
from threading import Thread,local
from queue import Queue

idx = list(range(1000))
fast = []

q = Queue(maxsize=0)

for i in idx:
    q.put(i)

thread_local = local()

def get_session() -> Session:
    if not hasattr(thread_local,'session'):
        thread_local.session = requests.Session()
    return thread_local.session

def download_link():
    session = get_session()
    while True:
        index = q.get()
        url = df_en.iloc[index]['image_url']
        with session.get(url, headers={'User-Agent': 'Mozilla/5.0'}) as response:
            if response.status_code == 200:
                fast.append({'image_url' : url, 'caption_description' : df_en.iloc[index]['caption_description']})
        q.task_done()

def download_all():
    thread_num = 10
    for _ in range(thread_num):
        t_worker = Thread(target=download_link)
        t_worker.start()
    q.join()


download_all()

In [8]:
len(fast)

979